In [39]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Baixar recursos adicionais do NLTK
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Carregar os dados
data = pd.read_excel('Expanded_Dataset_Tweets.xlsx')

# Pré-processamento do texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    processed_text = ' '.join(tokens)
    return processed_text

data['processed_tweets'] = data['tweets'].apply(preprocess_text)

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['processed_tweets'], data['classificador'], test_size=0.55, random_state=42)

# Criar o pipeline de classificação
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

# Definir os hiperparâmetros para o GridSearch
param_grid = {
    'tfidf__max_df': [0.8, 0.9, 1.0],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'clf__C': [0.1, 1, 10]
}

# Realizar o GridSearch com validação cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Imprimir os melhores hiperparâmetros
print(f"Melhores hiperparâmetros: {grid_search.best_params_}")

# Avaliar o modelo com os melhores hiperparâmetros
best_pipeline = grid_search.best_estimator_
predictions = best_pipeline.predict(X_test)
print(classification_report(y_test, predictions))

# URL da sua API
login_url = 'http://localhost:3000/auth/login'
# Credenciais do usuário
credentials = {
    'usuario': 'admin',
    'senha': 'admin'
}

# Fazendo uma solicitação POST para obter o token JWT
login_response = requests.post(login_url, json=credentials)
token = login_response.json()['token']
# Incluindo o token JWT no cabeçalho Authorization
headers = {
    'Authorization': "Bearer " + token
}

# Função para classificar tweets e atualizar no banco de dados
def classify_and_update_tweets(api_url):
    # Obter usuários da API
    response = requests.get(f"{api_url}/usuarios", headers=headers)

    if response.status_code == 200:
        try:
            usuarios = response.json()
        except ValueError:
            print("Erro ao decodificar JSON. Resposta não está no formato JSON.")
            return

        for usuario in usuarios:
            for perfil in usuario.get('perfis', []):
                for tweet in perfil.get('tweets', []):
                    tweet_text = tweet['texto']
                    processed_text = preprocess_text(tweet_text)
                    is_suicida = best_pipeline.predict([processed_text])[0] == 'Post  com Potencial Suicida'

                    # Atualizar o campo isSuicida no tweet
                    tweet_id = tweet['_id']
                    update_url = f"{api_url}/usuarios/update-tweet"
                    
                    # Incluir _id do usuário e nome do perfil
                    update_data = {
                        '_id': usuario['_id'],
                        'perfil_nome': perfil['usuario'],
                        'tweet_id': tweet_id,
                        'isSuicida': is_suicida,
                        'texto': tweet['texto'],
                        'link': tweet['link']
                    }
                    update_response = requests.put(update_url, json=update_data, headers=headers)

                    if update_response.status_code == 200:
                        print(f"Tweet {tweet_id} atualizado com sucesso.")
                    else:
                        print(f"Erro ao atualizar o tweet {tweet_id}. Código de status:", update_response.status_code)
    else:
        print("Erro ao acessar a API. Código de status:", response.status_code)

# Definir URL da API
api_url = "http://localhost:3000"

# Classificar e atualizar tweets
classify_and_update_tweets(api_url)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Melhores hiperparâmetros: {'clf__C': 1, 'tfidf__max_df': 0.8, 'tfidf__ngram_range': (1, 1)}
                             precision    recall  f1-score   support

Post  com Potencial Suicida       0.94      0.92      0.93       264
           Post Não Suicida       0.92      0.94      0.93       270

                   accuracy                           0.93       534
                  macro avg       0.93      0.93      0.93       534
               weighted avg       0.93      0.93      0.93       534

Tweet 66787f74cf15195e34aeb86c atualizado com sucesso.
Tweet 66787f74cf15195e34aeb86d atualizado com sucesso.
Tweet 66787f74cf15195e34aeb86e atualizado com sucesso.
Tweet 66787f74cf15195e34aeb86f atualizado com sucesso.
Tweet 66787f74cf15195e34aeb870 atualizado com sucesso.
Tweet 66787f74cf15195e34aeb871 atualizado com sucesso.
Tweet 66787f74cf15195e34aeb872 atualizado com sucesso.
Tweet 66787f74cf15195e34aeb873 atualizado com sucesso.
Tweet 66787f74cf15195e34aeb874 atualizado com suces

NameError: name 'response' is not defined